In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model("deepseek-r1:32b", model_provider="ollama")
model = init_chat_model("llama3.3", model_provider="ollama")


In [ ]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

In [ ]:
model.invoke([HumanMessage(content="What's my name?")])

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc123"}}

In [ ]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

In [ ]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

# Async

In [6]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage

model = init_chat_model("deepseek-r1:32b", model_provider="ollama")
model = init_chat_model("llama3.3", model_provider="ollama")

config = {"configurable": {"thread_id": "abc123"}}


In [7]:
from langgraph.graph import START, MessagesState, StateGraph
from langgraph.checkpoint.memory import MemorySaver

# Async function for node:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}


# Define graph as before:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
aapp = workflow.compile(checkpointer=MemorySaver())



In [8]:
query = "Hi! I'm Bob."
input_messages = [HumanMessage(query)]

# Async invocation:
output = await aapp.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()


================================== Ai Message ==================================

Hello Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?


In [9]:

query = "What's my name?"
input_messages = [HumanMessage(query)]

# Async invocation:
output = await aapp.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob! You told me that when we started talking.


# Adding a System Message

In [10]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [11]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [12]:
query = "Hi! I'm Bob."
input_messages = [HumanMessage(query)]

# Async invocation:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Arrrr, greetings to ye, Bob me hearty! 'Tis a grand day to be havin' a chat, don't ye think? What be bringin' ye to these fair waters? Are ye lookin' fer treasure, or just passin' the time with a swashbucklin' pirate like meself?


# Trimming

In [13]:
from langchain.chat_models import init_chat_model

model = init_chat_model("deepseek-r1:32b", model_provider="ollama")
model = init_chat_model("llama3.3", model_provider="ollama")

In [14]:
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage, trim_messages
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import START, MessagesState, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langgraph.graph.message import add_messages


trimmer = trim_messages(
    max_tokens=6_500_000,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)


workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke({"messages": trimmed_messages})
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3
import aiosqlite


db_path = 'checkpoints.db'
conn = sqlite3.connect(db_path, check_same_thread=False)

#memory = SqliteSaver(conn)
# Does not work
#memory = AsyncSqliteSaver(conn)


memory = MemorySaver()
app = workflow.compile(checkpointer=memory)
        
print(type(app))

<class 'langgraph.graph.state.CompiledStateGraph'>


In [15]:
from pprint import pprint

config = {"configurable": {"thread_id": "abc12"}}
inputs = {"messages": [HumanMessage(content="What is 1+1?")]}

async with AsyncSqliteSaver.from_conn_string(db_path) as saver:
    graph = workflow.compile(checkpointer=saver)
    async for event in graph.astream_events(inputs, config, version="v1"):
        pprint(event)

/Users/dan/miniforge3/envs/deep-dive/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


{'data': {'input': {'messages': [HumanMessage(content='What is 1+1?', additional_kwargs={}, response_metadata={})]}},
 'event': 'on_chain_start',
 'metadata': {'thread_id': 'abc12'},
 'name': 'LangGraph',
 'parent_ids': [],
 'run_id': '832e0ffc-711f-47c2-b70b-826e0fac9dda',
 'tags': []}
{'data': {'input': {'messages': [HumanMessage(content='What is 1+1?', additional_kwargs={}, response_metadata={})]}},
 'event': 'on_chain_start',
 'metadata': {'langgraph_checkpoint_ns': '__start__:4d6969a2-6e7a-92e2-0ff1-3eb69e9042c9',
              'langgraph_node': '__start__',
              'langgraph_path': ('__pregel_pull', '__start__'),
              'langgraph_step': 29,
              'langgraph_triggers': ['__start__'],
              'thread_id': 'abc12'},
 'name': '__start__',
 'parent_ids': [],
 'run_id': 'ba67e4b3-48d7-480a-96ec-88961814c733',
 'tags': ['graph:step:29', 'langsmith:hidden']}
{'data': {'input': {'messages': [HumanMessage(content='What is 1+1?', additional_kwargs={}, response_m

NotImplementedError in LogStreamCallbackHandler.on_llm_end callback: NotImplementedError('Trying to load an object that doesn\'t implement serialization: {\'lc\': 1, \'type\': \'not_implemented\', \'id\': [\'ollama\', \'_types\', \'Message\'], \'repr\': "Message(role=\'assistant\', content=\'\', images=None, tool_calls=None)"}')


{'data': {'chunk': AIMessageChunk(content='!', additional_kwargs={}, response_metadata={}, id='run-fdfa336a-d905-4385-88fc-2bac4b447c12')},
 'event': 'on_chat_model_stream',
 'metadata': {'checkpoint_ns': 'model:108eaa4c-3893-47ed-7446-83f819c78a02',
              'langgraph_checkpoint_ns': 'model:108eaa4c-3893-47ed-7446-83f819c78a02',
              'langgraph_node': 'model',
              'langgraph_path': ('__pregel_pull', 'model'),
              'langgraph_step': 30,
              'langgraph_triggers': ['start:model'],
              'ls_model_name': 'llama3.3',
              'ls_model_type': 'chat',
              'ls_provider': 'ollama',
              'ls_temperature': None,
              'thread_id': 'abc12'},
 'name': 'ChatOllama',
 'parent_ids': [],
 'run_id': 'fdfa336a-d905-4385-88fc-2bac4b447c12',
 'tags': ['seq:step:1']}
{'data': {'chunk': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'model': 'llama3.3', 'created_at': '2025-02-20T17:02:12.637281Z', 'done

In [16]:
# THIS COULD BE THE ONE I NEED

from pprint import pprint

config = {"configurable": {"thread_id": "abc12"}}
inputs = {"messages": [HumanMessage(content="What is my name?")]}

async with AsyncSqliteSaver.from_conn_string(db_path) as saver:
    graph = workflow.compile(checkpointer=saver)        
    async for chunk, metadata in graph.astream(
        {"messages": input_messages},
        config,
        stream_mode="messages",
    ):
        if isinstance(chunk, AIMessage):  # Filter to just model responses
            print(chunk.content, end="")

Fifth time's the charm, Bob! Yes, your name is indeed still Bob. I'm starting to think you're just making sure I remember! Don't worry, I've got it: your name is Bob!

In [ ]:
config = {"configurable": {"thread_id": "abc12"}}

query = "Hi! I'm Bob."
input_messages = [HumanMessage(query)]

# Async invocation:
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

In [ ]:
query = "What is my name"
input_messages = [HumanMessage(query)]

async for chunk, metadata in app.astream(
    {"messages": input_messages},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

In [ ]:
import asyncio

from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver
from langgraph.graph import StateGraph

builder = StateGraph(int)
builder.add_node("add_one", lambda x: x + 1)
builder.set_entry_point("add_one")
builder.set_finish_point("add_one")
async with AsyncSqliteSaver.from_conn_string("checkpoints.db") as memory:
    graph = builder.compile(checkpointer=memory)
    coro = graph.ainvoke(1, {"configurable": {"thread_id": "thread-1"}})
    print(asyncio.run(coro))

# Async history